**Project9**

Leader

m5271046 Yuki Tome

Member

m5271022 Yusuke Matsushima

m5271043 Hiroshi Tatsuta

m5271009 Shota Higa

Part 1: Understanding the GAN Algorithm

Generative Adversarial Networks (GANs) consist of two main components: a generator and a discriminator. The generator creates new data instances, while the discriminator evaluates them for authenticity. The discriminator also checks parts of the real data set to ensure its accuracy. The goal of the generator is to generate passable data to fool the discriminator, and the discriminator's goal is to identify the generator's fakes.

Here are the steps of the GAN algorithm:

Initialize: Initialize the generator and discriminator networks. These can be any kind of neural network, but they are often Convolutional Neural Networks (CNNs).

Generate Fake Data: The generator takes in random numbers and returns data (like an image). This data is called "fake".

Discrimination: The discriminator takes in both real data (actual images from the dataset) and fake data (images produced by the generator). It returns probabilities, a number between 0 and 1, with 1 representing a prediction of authenticity and 0 representing fake.

Calculate Loss: Use the discriminator's probabilities to calculate the loss for both the generator and the discriminator. The generator's loss is calculated using the probabilities for the fake images, while the discriminator's loss is calculated using the probabilities for both the real and fake images.

Backpropagation and Optimization: Use the gradients of the loss to update the generator and discriminator with gradient descent.

Repeat: Repeat steps 2-5 for a certain number of epochs, or until the discriminator and generator loss does not improve significantly.

Part 2: Implementing a GAN Program

I can provide you with a high-level overview of how to implement a GAN in Python using TensorFlow and Keras. However, due to the complexity of the code, I recommend following the detailed tutorials you provided for a step-by-step guide.

Here is a high-level overview:

Import Necessary Libraries: Import TensorFlow, Keras, and other necessary libraries.

Load and Preprocess Data: Load your dataset (like MNIST) and preprocess it. This usually involves normalizing the data and reshaping it.

Define the Generator: This is a neural network that takes in a random noise signal and outputs an image. It usually involves a series of dense and reshape layers.

Define the Discriminator: This is a neural network that takes in an image and outputs a probability that the image is real. It usually involves a series of convolutional layers.

Define the GAN: This involves connecting the generator and discriminator networks. You will also define the loss function and optimizers for both networks.

Train the GAN: This involves generating images with the generator, training the discriminator with both real and fake images, and then training the generator to try to fool the discriminator.

Part 3: Evaluating the Program

After training the GAN, you can evaluate it by generating new images and visually inspecting them. You can also compare the GAN's performance to other generative models like Autoencoders (AE) and Restricted Boltzmann Machines (RBM).

Here are some differences between these models:

GANs generate new data instances that are similar to your training data. They are great for generating new images, music, and even text.

Autoencoders are used for learning efficient codings of input data. They can be used for dimensionality reduction, or for learning to reconstruct input data. However, they are not explicitly designed to generate new

In [3]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Reshape, LeakyReLU, Dropout, Input
from keras.optimizers import Adam
from keras import initializers
from tqdm import tqdm

2023-07-05 10:23:28.952411: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5)/127.5
X_train = X_train.reshape(60000, 784)

In [5]:
random_dim = 100

def create_generator():
    generator = Sequential()
    generator.add(Dense(256, input_dim=random_dim, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    generator.add(LeakyReLU(0.2))
    generator.add(Dense(512))
    generator.add(LeakyReLU(0.2))
    generator.add(Dense(1024))
    generator.add(LeakyReLU(0.2))
    generator.add(Dense(784, activation='tanh'))
    return generator

In [6]:
def create_discriminator():
    discriminator = Sequential()
    discriminator.add(Dense(1024, input_dim=784, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    discriminator.add(Dense(512))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    discriminator.add(Dense(256))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    discriminator.add(Dense(1, activation='sigmoid'))
    return discriminator

In [7]:
def create_gan(discriminator, random_dim, generator):
    discriminator.trainable = False
    gan_input = Input(shape=(random_dim,))
    x = generator(gan_input)
    gan_output = discriminator(x)
    gan = Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=Adam())
    return gan

In [8]:
def train_gan(gan, discriminator, generator, random_dim, epochs=1, batch_size=128):
    for e in range(1, epochs+1):
        for _ in tqdm(range(batch_size)):
            noise = np.random.normal(0, 1, size=[batch_size, random_dim])
            image_batch = X_train[np.random.randint(0, X_train.shape[0], size=batch_size)]

In [ ]:
# Generate fake MNIST images
generated_images = create_generator().predict(noise)
X = np.concatenate([image_batch, generated_images])

# Labels for generated and real data
y_dis = np.zeros(2*batch_size)
# One-sided label smoothing
y_dis[:batch_size] = 0.9

# Train discriminator
discriminator.trainable = True
discriminator.train_on_batch(X, y_dis)

# Train generator
noise = np.random.normal(0, 1, size=[batch_size, random_dim])
y_gen = np.ones(batch_size)
discriminator.trainable = False
gan.train_on_batch(noise, y_gen)

if e == 1 or e % 20 == 0:
    print(f"Epoch {e} of {epochs}")

